

 Download necessary NLTK data



In [8]:
import nltk
import random
import time
from collections import Counter, defaultdict
from nltk.corpus import brown

# Download necessary NLTK data
nltk.download('brown')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Part 1: Corpus and Model Preparation

In [9]:
words = brown.words()
words = [word.lower() for word in words if word.isalpha()]
vocab = set(words)
unigram_freq = Counter(words)


bigrams = list(nltk.bigrams(words))
bigram_freq = Counter(bigrams)


def bigram_prob(prev, word):
    bigram_count = bigram_freq[(prev, word)]
    prev_count = unigram_freq[prev]
    if prev_count == 0:
        return 0
    return bigram_count / prev_count

Part 2: Candidate Generation Methods

In [10]:
def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

delete_dict = defaultdict(list)

for word in vocab:
    splits = [(word[:i], word[i:]) for i in range(len(word))]
    deletes = [L + R[1:] for L, R in splits if R]
    for d in deletes:
        delete_dict[d].append(word)

def candidates_symmetric(word):
    splits = [(word[:i], word[i:]) for i in range(len(word))]
    deletes = [L + R[1:] for L, R in splits if R]
    candidates = set()
    for d in deletes:
        candidates.update(delete_dict.get(d, []))
    return candidates


Part 3: Spelling Correction Logic

In [11]:
def correct_nonword(word, method='A'):
    if method == 'A':
        cands = edits1(word)
    else:
        cands = candidates_symmetric(word)
    valid_cands = [w for w in cands if w in vocab]
    if not valid_cands:
        return word
    return max(valid_cands, key=lambda w: unigram_freq[w])


def correct_realword(prev_word, word, method='A'):
    if method == 'A':
        cands = edits1(word)
    else:
        cands = candidates_symmetric(word)
    valid_cands = [w for w in cands if w in vocab]
    valid_cands.append(word)
    best_candidate = word
    max_prob = bigram_prob(prev_word, word)
    for cand in valid_cands:
        prob = bigram_prob(prev_word, cand)
        if prob > max_prob:
            max_prob = prob
            best_candidate = cand
    return best_candidate

Part 4: Evaluation

In [12]:
sentences = list(brown.sents())
random.shuffle(sentences)
test_size = int(0.1 * len(sentences))
test_sentences = sentences[:test_size]

def introduce_error(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    if len(word) == 0:
        return word
    i = random.randint(0, len(word) - 1)
    c = random.choice(letters)
    error_type = random.choice(['delete', 'replace', 'insert', 'transpose'])
    if error_type == 'delete' and len(word) > 1:
        return word[:i] + word[i+1:]
    elif error_type == 'replace':
        return word[:i] + c + word[i+1:]
    elif error_type == 'insert':
        return word[:i] + c + word[i:]
    elif error_type == 'transpose' and len(word) > 1 and i < len(word) - 1:
        return word[:i] + word[i+1] + word[i] + word[i+2:]
    return word

nonword_tests = []
realword_tests = []

for sent in test_sentences:
    if len(sent) < 2:
        continue
    idx = random.randint(0, len(sent) - 1)
    original_word = sent[idx].lower()
    if not original_word.isalpha():
        continue
    error_word = introduce_error(original_word)

    if error_word not in vocab:
        corrupted = sent[:idx] + [error_word] + sent[idx+1:]
        nonword_tests.append((sent, corrupted, idx))

    elif error_word in vocab and error_word != original_word:
        corrupted = sent[:idx] + [error_word] + sent[idx+1:]
        realword_tests.append((sent, corrupted, idx))


def evaluate_nonword(method='A'):
    correct = 0
    for original, corrupted, idx in nonword_tests:
        word = corrupted[idx].lower()
        corrected = correct_nonword(word, method)
        if corrected == original[idx].lower():
            correct += 1
    return correct / len(nonword_tests)

def evaluate_realword(method='A'):
    correct = 0
    for original, corrupted, idx in realword_tests:
        prev_word = corrupted[idx-1].lower() if idx > 0 else '<s>'
        word = corrupted[idx].lower()
        corrected = correct_realword(prev_word, word, method)
        if corrected == original[idx].lower():
            correct += 1
    return correct / len(realword_tests)


def compare_methods():
    start_a = time.time()
    evaluate_nonword(method='A')
    time_a = time.time() - start_a

    start_b = time.time()
    evaluate_nonword(method='B')
    time_b = time.time() - start_b

    print(f"Method A took {time_a:.4f} seconds")
    print(f"Method B took {time_b:.4f} seconds")
    faster = 'A' if time_a < time_b else 'B'
    print(f"Method {faster} is faster")


Running the evaluation

In [13]:
print("Evaluating Non-Word Errors:")
accuracy_a = evaluate_nonword(method='A')
accuracy_b = evaluate_nonword(method='B')
print(f"Method A Accuracy: {accuracy_a*100:.2f}%")
print(f"Method B Accuracy: {accuracy_b*100:.2f}%")

print("\nEvaluating Real-Word Errors:")
accuracy_real = evaluate_realword(method='A')  # using method A for real-word
print(f"Real-Word Accuracy: {accuracy_real*100:.2f}%")

print("\nComparing Performance for Non-Word Errors:")
compare_methods()

Evaluating Non-Word Errors:
Method A Accuracy: 83.67%
Method B Accuracy: 39.13%

Evaluating Real-Word Errors:
Real-Word Accuracy: 60.89%

Comparing Performance for Non-Word Errors:
Method A took 0.3748 seconds
Method B took 0.0490 seconds
Method B is faster


Example test sentences

In [14]:
example_sentences = [
    "I hav a good feeling about this.",
    "This is a test sentnce.",
    "I would like to sea the world.",
    "Please meat me at the station."
]

print("\nExample Corrections:")
for sent in example_sentences:
    tokens = nltk.word_tokenize(sent)
    corrected = []
    for i, word in enumerate(tokens):
        word_lower = word.lower()
        if word_lower not in vocab:
            corrected_word = correct_nonword(word_lower, method='A')
            corrected.append(corrected_word)
        else:
            prev = tokens[i-1].lower() if i > 0 else '<s>'
            corrected_word = correct_realword(prev, word_lower, method='A')
            corrected.append(corrected_word)
    print("Original:", sent)
    print("Corrected:", " ".join(corrected))
    print()


Example Corrections:
Original: I hav a good feeling about this.
Corrected: i had a good feeling about his a

Original: This is a test sentnce.
Corrected: this is a test sentence a

Original: I would like to sea the world.
Corrected: i could like to see the world a

Original: Please meat me at the station.
Corrected: please meat me a the station a

